In [0]:
from pyspark.sql.functions import col, avg, count, max, min, stddev, round, desc, when

df_silver = spark.table("nttdataeducacao.silver.resultados_2024")
display(df_silver)

In [0]:
    df_gold_estados = df_silver.groupBy("exam_state") \
    .agg(
        count("candidate_id").alias("total_inscritos"),
        count(when(col("is_present_all_exams") == True, 1)).alias("total_presentes_todos_dias"),
        round(avg("score_total"), 2).alias("media_geral"),
        round(avg("score_math"), 2).alias("media_matematica"),
        round(avg("score_essay"), 2).alias("media_redacao"),
        round(avg("score_humanities"), 2).alias("media_humanas"),
        round(avg("score_nature"), 2).alias("media_natureza"),
        round(avg("score_languages"), 2).alias("media_linguagens"),
        round(stddev("score_total"), 2).alias("desvio_padrao_notas"),
        max("score_total").alias("nota_maxima_estado")
    ) \
    .orderBy(desc("media_geral"))

display(df_gold_estados)

df_gold_estados.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("nttdataeducacao.gold.agg_performance_estado")

In [0]:
display("nttdataeducacao.silver.resultados_2024")

In [0]:
df_gold_escolas = df_silver \
    .filter(col("school_id") != -1) \
    .groupBy("school_id", "school_city", "school_state", "school_type_id") \
    .agg(
        count("candidate_id").alias("qtd_alunos"),
        round(avg("score_total"), 2).alias("media_escola_total"),
        round(avg("score_essay"), 2).alias("media_escola_redacao"),
        round(avg("score_math"), 2).alias("media_escola_matematica"),
        round(avg("score_languages"), 2).alias("media_escola_linguagens"),
        round(avg("score_humanities"), 2).alias("media_escola_humanas"),   
        round(avg("score_nature"), 2).alias("media_escola_natureza")
    ) \
    .filter(col("qtd_alunos") >= 10) \
    .orderBy(desc("media_escola_total"))

display(df_gold_escolas)

df_gold_escolas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("nttdataeducacao.gold.agg_ranking_escolas")

In [0]:
df_gold_dependencia = df_silver \
    .filter(col("school_id") != -1) \
    .groupBy("school_type_id") \
    .agg(
        count("candidate_id").alias("qtd_alunos"),
        round(avg("score_total"), 2).alias("media_geral"),
        round(avg("score_essay"), 2).alias("media_redacao"),
        round(avg("score_math"), 2).alias("media_matematica"),
        round(avg("score_languages"), 2).alias("media_escola_linguagens"),
        round(avg("score_humanities"), 2).alias("media_escola_humanas"),   
        round(avg("score_nature"), 2).alias("media_escola_natureza")
    ) \
    .withColumn(
        "tipo_escola",
        when(col("school_type_id") == 1, "Federal")
        .when(col("school_type_id") == 2, "Estadual")
        .when(col("school_type_id") == 3, "Municipal")
        .when(col("school_type_id") == 4, "Privada")
        .otherwise("Outros")
    ) \
    .orderBy(col("media_geral").desc())

display(df_gold_dependencia)

df_gold_dependencia.write \
    .format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable("nttdataeducacao.gold.agg_publica_vs_privada")